In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import StackingClassifier


In [24]:
df = pd.read_csv('iith_foml_2020_train.csv')
df.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24,Target Variable (Discrete)
0,1404,12,64,14,3,1,1,1,110.502,35775.2,...,NaN,NaN,15.04,104,12,2,32,1409,37677.1,1
1,909,0,235,32,1,1,1,1,-40.448,35779.4,...,2200.3,4900.005,12.03,20,1,0,13,909,25239.1,1
2,654,3,175,2,1,1,1,1,-27.445,35770.4,...,1973.3,10000.004,13.01,1,1,0,13,654,27683.5,1
3,1372,12,382,14,2,0,1,0,0.001,509.2,...,NaN,NaN,NaN,313,12,10,54,1377,39363.2,0
4,786,3,199,2,1,0,1,0,0.001,612.1,...,NaN,NaN,NaN,171,1,5,11,786,40044.4,2


In [25]:
df_tgt = df['Target Variable (Discrete)']
df.drop(columns = 'Target Variable (Discrete)', inplace = True)
df_tgt.shape

(994,)

In [4]:
df_tst = pd.read_csv('iith_foml_2020_test.csv')
df_tst.head()

,Feature 1 (Discrete),Feature 2 (Discrete),Feature 3 (Discrete),Feature 4 (Discrete),Feature 5 (Discrete),Feature 6 (Discrete),Feature 7 (Discrete),Feature 8 (Discrete),Feature 9,Feature 10,...,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19 (Discrete),Feature 20 (Discrete),Feature 21 (Discrete),Feature 22 (Discrete),Feature 23 (Discrete),Feature 24
0,146,12,42,14,7,1,1,1,118.004,35693.5,...,4200.3,1900.1,NaN,NaN,44,12,2,42,146,37384.5
1,35,0,12,5,0,0,1,0,0.001,471.5,...,531.4,NaN,NaN,NaN,17,6,8,0,35,41465.1
2,1018,8,259,2,1,1,1,1,NaN,35774.5,...,5514.2,NaN,NaN,15.04,1,1,3,20,1018,37826.2
3,383,7,117,5,1,1,1,1,53.002,34094.1,...,3358.4,1100.2,14000.001,15.04,101,6,3,20,383,40277.3
4,1216,7,40,5,2,0,1,4,0.005,1471.3,...,225.1,NaN,NaN,3.02,276,6,7,43,1221,28419.5


In [26]:
df2 = df
for column in df2.columns:
    if(column.endswith('(Discrete)')):
        df2[column].fillna(df[column].mode(), inplace = True)
    else:
        df2[column].fillna(df[column].median(), inplace = True)
df2.shape

(994, 24)

In [6]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=25, random_state=42)),
    ('xgb', xgb.XGBClassifier(objective='multi:softmax', n_estimators = 25, random_state=42))
]

In [7]:
clf = StackingClassifier(estimators=estimators)
model = clf.fit(df2.values, df_tgt.values)

/home/ashutosh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:37:01] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (17) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of cla

[17:37:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (17) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '


[17:37:02] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:37:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:37:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:37:03] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the ol

In [8]:
accuracy_score(df_tgt.values, model.predict(df.values))

0.9688128772635815

In [12]:
for column in df_tst.columns:
    if(column.endswith('(Discrete)')):
        df_tst[column].fillna(df[column].mode(), inplace = True)
    else:
        df_tst[column].fillna(df[column].median(), inplace = True)
df_tst.shape

(426, 24)

In [13]:
p = model.predict(df_tst.values).astype('int')
p.shape

(426,)

In [14]:
np.unique(p, return_counts=True)

(array([ 0,  1,  2,  5,  6, 14]), array([106, 225,  50,  16,  27,   2]))

In [15]:
basic = pd.read_csv('~/Documents/Current_Courses/FML/Hackathon/basic/basic.csv')

In [18]:
np.sum(np.abs(basic.values[:,1]-p))

32

In [43]:
estimators = [
    ('rf', RandomForestClassifier(n_estimators=100, subsample = 0.5)),
    ('xgb', xgb.XGBClassifier(objective='multi:softmax', n_estimators = 100, subsample = 0.5))
]

TypeError: __init__() got an unexpected keyword argument 'subsample'

In [44]:
clf = StackingClassifier(estimators=estimators)
model2 = clf.fit(df2.values, df_tgt.values)

/home/ashutosh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:44:46] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (17) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '
/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of cla

[17:44:49] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (17) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '


[17:44:50] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/home/ashutosh/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:901: RuntimeWarning: Number of classes in training fold (16) does not match total number of classes (18). Results may not be appropriate for your use case. To fix this, use a cross-validation technique resulting in properly stratified folds
  warnings.warn('Number of classes in training fold ({}) does '


[17:44:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:51] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:44:52] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [45]:
accuracy_score(df_tgt.values, model2.predict(df2.values))

0.9708249496981891

In [46]:
p2 = model.predict(df_tst.values).astype('int')
p2.shape

(426,)

In [47]:
np.unique(p2, return_counts=True)

(array([ 0,  1,  2,  5,  6, 14]), array([106, 225,  50,  16,  27,   2]))

In [48]:
np.sum(np.abs(basic.values[:,1]-p2))

32

In [49]:
df_tst['Category'] = p
df_tst['Id'] = df_tst.index + 1

In [50]:
df_tst[['Id', 'Category']].to_csv("stacking1.csv", index = False)

In [51]:
np.unique(df_tst['Category'].values, return_counts=True)

(array([ 0,  1,  2,  5,  6, 14]), array([106, 225,  50,  16,  27,   2]))

In [52]:
df_tst[['Id', 'Category']]

,Id,Category
0,1,6
1,2,2
2,3,1
3,4,1
4,5,1
...,...,...
421,422,1
422,423,1
423,424,1
424,425,1
